针对MarketBasket数据集进行购物篮分析（频繁项集及关联规则挖掘）	https://www.kaggle.com/dragonheir/basket-optimisation			"数据预处理（10points）
完成代码（20points）
结果正确（20points）"

In [1]:
import numpy as np
import pandas as pd

In [2]:
datas = pd.read_csv("Market_Basket_Optimisation.csv", header=None)

In [3]:
datas.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


方法1、采用efficient_apriori工具包

In [4]:
transaction1 = []
x, y = datas.shape
for i in range(0, x):
    temp = []
    for j in range(0, y):
        if str(datas.values[i, j]) != 'nan':
            temp.append(str(datas.values[i, j]))
    transaction1.append(temp)

In [5]:
from efficient_apriori import apriori

In [6]:
itemsets, rules = apriori(transaction1, min_support=0.02, min_confidence=0.4)

In [7]:
print("频繁项集:{}".format(itemsets))

频繁项集:{1: {('low fat yogurt',): 574, ('almonds',): 153, ('salmon',): 319, ('green tea',): 991, ('vegetables mix',): 193, ('frozen smoothie',): 475, ('mineral water',): 1788, ('tomato juice',): 228, ('olive oil',): 494, ('avocado',): 250, ('energy drink',): 200, ('cottage cheese',): 239, ('honey',): 356, ('shrimp',): 536, ('burgers',): 654, ('eggs',): 1348, ('meatballs',): 157, ('turkey',): 469, ('whole wheat rice',): 439, ('milk',): 972, ('energy bar',): 203, ('french fries',): 1282, ('whole wheat pasta',): 221, ('soup',): 379, ('frozen vegetables',): 715, ('spaghetti',): 1306, ('cookies',): 603, ('cooking oil',): 383, ('champagne',): 351, ('oil',): 173, ('chicken',): 450, ('chocolate',): 1229, ('fresh tuna',): 167, ('tomatoes',): 513, ('red wine',): 211, ('pepper',): 199, ('pancakes',): 713, ('ham',): 199, ('grated cheese',): 393, ('fresh bread',): 323, ('ground beef',): 737, ('escalope',): 595, ('herb & pepper',): 371, ('strawberries',): 160, ('cake',): 608, ('hot dogs',): 243, ('brow

In [8]:
print("关联规则:{}".format(rules))

关联规则:[{olive oil} -> {mineral water}, {ground beef} -> {mineral water}, {soup} -> {mineral water}]


2、采用mlxtend.frequent_patterns工具包

In [9]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [10]:
# 由于mlxtend的模型只接受特定的数据格式。（TransactionEncoder类似于独热编码，每个值转换为一个唯一的bool值）
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

In [11]:
data_arr = te.fit_transform(transaction1)

In [12]:
data_pd = pd.DataFrame(data=data_arr,columns=te.columns_)

In [13]:
data_pd[:5]

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [14]:
itemset1 = apriori(data_pd, min_support=0.02, use_colnames=True)
# use_colnames=True表示使用元素名字，默认的False使用列名代表元素

In [15]:
rules1 = association_rules(itemset1, metric='lift', min_threshold=1)

In [16]:
# 关联规则
itemset1

,support,itemsets
0,0.020397,(almonds)
1,0.033329,(avocado)
2,0.033729,(brownies)
3,0.087188,(burgers)
4,0.030129,(butter)
...,...,...
98,0.020131,"(whole wheat rice, mineral water)"
99,0.022930,"(olive oil, spaghetti)"
100,0.025197,"(pancakes, spaghetti)"
101,0.021197,"(shrimp, spaghetti)"


In [17]:
# 频繁项集
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(eggs),(burgers),0.179709,0.087188,0.028796,0.160237,1.837830,0.013128,1.086988
1,(burgers),(eggs),0.087188,0.179709,0.028796,0.330275,1.837830,0.013128,1.224818
2,(burgers),(french fries),0.087188,0.170911,0.021997,0.252294,1.476173,0.007096,1.108844
3,(french fries),(burgers),0.170911,0.087188,0.021997,0.128705,1.476173,0.007096,1.047650
4,(burgers),(mineral water),0.087188,0.238368,0.024397,0.279817,1.173883,0.003614,1.057552
...,...,...,...,...,...,...,...,...,...
89,(spaghetti),(pancakes),0.174110,0.095054,0.025197,0.144717,1.522468,0.008647,1.058066
90,(shrimp),(spaghetti),0.071457,0.174110,0.021197,0.296642,1.703760,0.008756,1.174209
91,(spaghetti),(shrimp),0.174110,0.071457,0.021197,0.121746,1.703760,0.008756,1.057260
92,(tomatoes),(spaghetti),0.068391,0.174110,0.020931,0.306043,1.757755,0.009023,1.190117
